In [163]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[autoreload of src.dim_reduction failed: Traceback (most recent call last):
  File "/home/bombra/anaconda3/envs/padme/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 273, in check
    superreload(m, reload, self.old_objects)
  File "/home/bombra/anaconda3/envs/padme/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 496, in superreload
    update_generic(old_obj, new_obj)
  File "/home/bombra/anaconda3/envs/padme/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/home/bombra/anaconda3/envs/padme/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    if update_generic(old_obj, new_obj):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/bombra/anaconda3/envs/padme/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 393, in update_generic
    update(a, b)
  File "/home/bombra/anaconda3/envs/padme/lib/python3.11/site-packages/

In [164]:
from yaml import safe_load, YAMLError
from src.snapshots import snapshots_assembly
from src.dim_reduction import SVD, AutoEncoder

In [165]:
with open("parameters.yaml", "r") as stream:
    try:
        params = safe_load(stream)
    except YAMLError as exc:
        print(exc)

In [166]:
params

{'snapshots': {'file_type_str': 'h5_fenics',
  'folder': 'data',
  'file_name_contains': ['concentration'],
  'dataset': None},
 'svd': {'trunc_basis': 10,
  'svd_type': 'randomized_svd',
  'power_iterations': 1,
  'oversampling': 20},
 'auto_encoder': {'batch_size': 10,
  'num_epochs': 10,
  'learning_rate': '1e-4',
  'weight_decay': '1e-8',
  'loss_function': 'smooth_l1_loss',
  'loss_parameters': {'beta': 0.2},
  'num_workers': 2,
  'number_of_hidden_layers': 5,
  'hidden_layers_sizes': [50, 25, 12, 6, 3],
  'hidden_layers_activation_function': ['leaky_relu',
   'leaky_relu',
   'leaky_relu',
   'leaky_relu',
   ''],
  'hidden_layers_activation_function_parameters': [0.2, 0.2, 0.2, 0.2, 'None'],
  'decoder_activation_function': 'sigmoid',
  'decoder_activation_function_parameter': 'None'}}

In [167]:
try:
    print(snapshots.shape)
except:
    filenames, snapshots = snapshots_assembly(params["snapshots"])

(70801, 2702)


In [168]:
# svd_params = {
#     "preprocessing_type": "min_max",
#     "trunc_basis": snapshots.shape[1],
#     "svd_type": "full_svd",  # ["full_svd", "randomized_svd"]
#     "power_iterations": 1,
#     "oversampling": 20,
# }

# svd_step = SVD(snapshots, svd_params)
# svd_step.fit()
# svd_step.s.shape

In [169]:
snapshots_sample = snapshots[:500, :100]
svd_step_randomized = SVD(snapshots_sample, params["svd"])
svd_step_randomized.fit()
svd_step_randomized.s.shape

(10,)

In [170]:
# import matplotlib.pyplot as plt

# plt.plot(svd_step_random.s[:220])
# plt.plot(svd_step.s[:220], "--")
# plt.show()

In [171]:
# dict_ae = {}

# ae_test = AutoEncoder(snapshots_sample, dict_ae)

In [172]:
# dict_ae = {
#     "batch_size": 10,
#     "num_epochs": 10,
#     "num_workers": 2,
#     "hidden_layer_size": 50,
#     "latent_size": 10,
# }

# ae_test_2 = AutoEncoderTrain(snapshots_sample, dict_ae)
# ae_test_2.fit()

In [173]:
# dict_ae = {
#     "batch_size": 10,
#     "num_epochs": 10,
#     "num_workers": 2,
#     "number_of_hidden_layers": 4,
#     "hidden_layers_sizes": [50, 25, 12, 6],
#     "hidden_layers_activation_function": ["leaky_relu", "leaky_relu", "leaky_relu", ""],
#     "hidden_layers_activation_function_parameters": [0.2, 0.2, 0.2, None],
#     "decoder_activation_function": "sigmoid",
#     "decoder_activation_function_parameter": None,
# }

In [174]:
type(snapshots_sample)

numpy.ndarray

In [175]:
ae_test_2 = AutoEncoder(snapshots_sample, params["auto_encoder"])
ae_test_2.fit()

print(ae_test_2.auto_encoder.encoder)
print(ae_test_2.auto_encoder.decoder)

<class 'torch.utils.data.dataloader.DataLoader'>


100%|██████████| 10/10 [00:03<00:00,  3.06it/s]

Sequential(
  (0): Linear(in_features=100, out_features=50, bias=True)
  (1): LeakyReLU(negative_slope=0.2)
  (2): Linear(in_features=50, out_features=25, bias=True)
  (3): LeakyReLU(negative_slope=0.2)
  (4): Linear(in_features=25, out_features=12, bias=True)
  (5): LeakyReLU(negative_slope=0.2)
  (6): Linear(in_features=12, out_features=6, bias=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Linear(in_features=6, out_features=3, bias=True)
)
Sequential(
  (0): Linear(in_features=3, out_features=6, bias=True)
  (1): LeakyReLU(negative_slope=0.2)
  (2): Linear(in_features=6, out_features=12, bias=True)
  (3): LeakyReLU(negative_slope=0.2)
  (4): Linear(in_features=12, out_features=25, bias=True)
  (5): LeakyReLU(negative_slope=0.2)
  (6): Linear(in_features=25, out_features=50, bias=True)
  (7): LeakyReLU(negative_slope=0.2)
  (8): Linear(in_features=50, out_features=100, bias=True)
  (9): Sigmoid()
)
